## Project Phase II ##
#### Kerwin Chen ####
__Research Question:__ How have song lyrics changed over the decades (1960s-2010s)? <br>
I'm hoping to analyze trends and changes in lyrics, and also to create a regression model that can predict which decade a song is from based on its lyrics. 

In [ ]:
import requests
import bs4
import numpy as np
import pandas as pd

There were two parts to my data collection. I first needed to scrape the names of songs of every year from 1960 to 2019 (data scraping pt 1), spanning a total of six decades (60s, 70s, 80s, 90s, 00s, 10s). The Billboard Hot 100 is a standard record chart in the music industry that tracks the most popular songs (through a point system that is a function of the song's sales), and release a year-end top 100 songs of the year every year. There is a website that's been keeping a record of these year-end hot 100s, which I decided to scrape. After scraping that, I used that as an input as a scraped Genius for lyrics via their API. Genius is an American digital media company that provides lyrics for many English and foreign language songs. 

### Data Collection Pt. I ###
Part one of data collection involves __scraping for the name of the song and its artist.__ On initial glance, the top 100 songs of each year seemed to be stored in an html table within appropriate tags and unique identifying tag attributes, all on a page with an easily generated URL. However, after initially naively scrapping every year from 1960 to 2019, I encountered multiple errors here and there. I decided to investigate each page more closely, and found out that the way the data was stored was grossly inconsistent across the years. I was hoping at a worst case scenario to need to scrape each decade individually, but it seems the differences were not dependent on decade. For example, in 2019 the data was stored within p tags, but 2018's data was each stored as an article element. Moreover, 2015 was stored within a table element, while 2013 didn't even have tags. This essentially forced me to check the html file for every single year by hand, and I kept an excel spreadsheet open to help me track how data is stored for each year. This made data collection part one take much much longer than anticipated. I was happy to find that there was consistency from 2012 to 1960 with data stored in rows of a table element, which meant that I could scrape that whole chunk together. For the exploratory analysis, I did not have time to scrap the years outside of 1960 to 2012, but I will in the future scrap 2013-2019 to complete the 2010s decade. 

In [ ]:
decade = []
year = []
artist = []
song = []
years = list(range(1960,2013))
for i in years:
    site = "http://billboardtop100of.com/" + str(i) + "-2/"
    webpage = requests.get(site)
    soup = bs4.BeautifulSoup(webpage.text, 'html.parser')
    table = soup.findAll('tr')
    decade += [int(str(i)[:-1] + "0")] * 100
    year += [i] * 100
    artist += [i.text.split("\n")[2] for i in table]
    song += [i.text.split("\n")[3] for i in table]
    #comment below is my final failed attempt at scraping the entire 1960-2019 chunk before noticing the inconsistency
    #artist += [i.text.strip() for i in table if "id" in i.attrs and not i.text.strip().isnumeric()]
    #song += [i.text.strip() for i in table if i.attrs == {} and not i.text.isnumeric()]
    print('Done with', i) #allowed me to track the progress of the scraping

In [ ]:
print(len(decade), len(year), len(artist), len(song))

As suspected, each year should have 100 songs, giving us 1000 songs per decade with $2012 + 1 - 1960 = 53$ total years, resulting in a total of 5300 songs.

In [ ]:
#1960-2012
top100 = pd.DataFrame({'decade': decade, 'year': year, 'artist':artist, 'song':song})
top100.to_csv('top100.csv') #stored data into dataframe and exported to csv

In [ ]:
top100.head()

In [ ]:
top100.tail()

### Data Collection Pt. II ###
The second part of my data collection involves taking the list of songs I previously scrapped, and __finding the lyrics for each song.__ To do this, I had to sign up on Genius and make an account to obtain a unique authorization key to access their API. A data scientist named John W. Miller wrote a python package called lyricsgenius that wraps the Genius API, making it easier to download the song lyrics. I used this to get the lyrics of all the songs that I scraped from 1960-2012. I used a try-except block to handle cases were the song lyrics weren't found. This process ended up taking almost four hours to complete.

In [ ]:
import lyricsgenius as lg
lyrics = []
genius = lg.Genius("JAlCQvWQxOy0Ertp8NhDj4wHzxBwc12vQiToA2HkRFqDHCLOBYTj0DjgrZCldg0f")
progress = 500
for i in range(len(top100.index)):
    locate_song = genius.search_song(top100["song"][i], top100["artist"][i])
    try:
        lyrics += [locate_song.lyrics]
    except AttributeError:
        lyrics += ["NONE"] #if error thrown (ie there is no lyrics), put "NONE" into the lyrics list
    if i % progress == 0:
        print("Done with", i, "songs") #allowed me to track the progress of the scraping
        progress += 500
print("Done with all songs")

In [ ]:
top100['lyrics'] = lyrics
top100.to_csv('top100Ly.csv')

In [ ]:
top100.head()

In [ ]:
top100.tail()

In [ ]:
no_lyrics = top100.loc[top100["lyrics"] == "NONE"].copy().groupby("decade")
no_lyrics.count()

I knew that there were going to be songs where I couldn't find the lyrics to, and I was initially worried that there would be a tendency for older songs to not have lyrics, giving me a smaller and smaller sample of songs as we go back in time. However, I displayed the number of songs with missing lyrics by decade, and every decade seemed to have about 10-20 songs missing. The 2010's so far only has one song missing, but that may also be because it currently only has songs from 2010-2012. I don't think the 10-20 missing song per decade was something I needed to worry about, because they represent a mere 1-2% of the 1000 songs per decade.

In [ ]:
from collections import Counter

This collections module has a Counter object that would allow me to quickly tally frequency of words within songs into dictionary-like objects.

In [ ]:
top100_bydec = top100.groupby(by='decade')
count_bydec = []
#top100_byyear.get_group(1960)["lyrics"]
for i in range(1960, 2020, 10)`: #looping through each decade
    dec_lyrics = top100_bydec.get_group(i)["lyrics"] #getting lyrics of each group
    count_bydec.append(Counter("".join(filter(lambda x: x not in [",", ".", "!", "?"], dec_lyrics)).lower().split()))
    #append to a new list a Counter object containing frequency of words

Because I wanted words like "Love", "love", and "love," to all count as the same word, I made sure to remove punctuation using the filter function and an anonymous function, and setting everything to lowercase letters, before I passed it into a Counter object. 

In [ ]:
len(count_bydec) #to confirm number of decades we have

In [ ]:
len(count_bydec)
x = 1960
for i in count_bydec:
    print("Decade", x, ":\n", i.most_common(10), "\n")
    x += 10

As suspected, the top 10 words of each decade weren't unique and insightful, as they were just common English words. I plan to remove these to find unique words of each decade.

In [ ]:
decade_counter = pd.DataFrame({'decade' : list(range(1960, 2020, 10)), 'count_bydec': count_bydec})
decade_counter.to_csv('count_bydec.csv')

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
keywords = ["love","baby","money", "gonna"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
plt.legend(keywords)
plt.title("Frequency of Select Generic Words in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

As a note, due to the 2010s having only 300 songs so far, its not very representative of that decade. I selected a few words that I noticed from personal experience showed up in songs frequently. As suspected, love is a common word found in songs of all decades, and the word _money_ having a peak during the 70's and _baby_ peaking in the 90's.

In [ ]:
keywords = ["phone", "radio", "stereo", "pager", "tape"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
plt.legend(keywords)
plt.title("Frequency of Select Technological Words in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

For this plot I selected key words related to technologies used in daily life and took a look at their frequency. Something interesting to note is the small peak during the 80's for the word _stereo_, and then a higher peak for the 2010's.  

In [ ]:
keywords = ["bitch","fuck", "hoes"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
censor = lambda x: x[0] + "*" + x[2:]
plt.legend([censor(i) for i in keywords])
plt.title("Frequency of Select Profanities/Derogatory Terms in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

When looking at frequency of profanities, we see a huge jump during the 2000s. I suspect that when I finish collecting data for the 2010s, I will see a similarly high peak. 

In [ ]:
keywords = ["sunshine", "moon", "river", "sun"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
plt.legend(keywords)
plt.title("Frequency of Select Nature Words in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

In the 80s we see a dip in words related to nature, and a general decrease since.

In [ ]:
keywords = ["disco", "party", "bar", "club", "dance"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
plt.legend(keywords)
plt.title("Frequency of Entertainment Related Words in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

The rise of _disco_ in the 70's is reflected in song lyrics of that time as we can see a small peak in our graph. We also see a large peak for _club_ and _party_ as we reach the 2000s. _Dance_ shows a large peak in both the 70's and 2000's. 

In [ ]:
keywords = ["girl", "boy", "woman", "man"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
plt.legend(keywords)
plt.title("Frequency of Select Gender Words in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

We can see from this graph that the most common method to refer to females seem to be _girl_, while for males, _man_. 

In [ ]:
keywords = ["ass", "butt", "bum", "bottom"]
for i in keywords:
    plt.plot(list(range(1960,2020,10)), [j[i] for j in count_bydec])
plt.legend(keywords)
plt.title("Frequency of variations of buttocks in Songs Over Time")
plt.xlabel("decade")
plt.ylabel("frequency")
plt.show()

### Data Description ###
__What are the observations (rows) and the attributes (columns)?__ <br>
The attributes were the decade, and the count of words by decade.
Each observation (row) represents a decade, and the count of words is stored as Counter objects.<br>
__What processes might have influenced what data was observed and recorded and what was not?__<br>
Because the creators of the data (Billboard) isn't involved with the music itself, I feel that it's able to maintain a pretty objective perspective because its ranking is based on a public function of music popularity and cumulative sales. <br>
__What preprocessing was done, and how did the data come to be in the form that you are using?__<br>
I obtained this dataset by first scraping for the top 100 songs of each year from Billboard Hot 100. Using this, I scraped for the lyrics for each of these songs, and then grouped them by decade before finding the frequency fo each word.  

### Data Limitations ###
One limitation of my data is its size. Although I have 1000 songs per decade, that's really only 100 songs a year, which I feel like could have an impact on the results, so it would have been better if I could obtain more songs per year. I'm also still missing data for 2013-2019, so my 2010's data is still not representative of the decade. 

### Question for Reviewers ###
My data is pretty qualitative, so I'm having trouble with coming up with other analysis ideas other than the ones I have with comparing word frequency over time. I plan to also create a regression model that can predict decade based on lyrics, but other than that, what other analysis would you suggest?

How can I clarify my research question and make it better?

Any other improvements/suggestions/words to take a look at are welcomed :)